In [1]:
import transformers
from script.tokenizer import KmerTokenizer

import evaluate
import pandas as pd
from tqdm import tqdm

import argparse
import os
import csv
import sys
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import json
import random
import re

import pandas as pd
import re
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, PreTrainedTokenizerFast, LineByLineTextDataset, BertConfig, AutoTokenizer, pipeline
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch

from sklearn.preprocessing import LabelEncoder
import numpy as np
from datasets import Dataset
import os

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.cuda.current_device(), device

(0, device(type='cuda'))

# data

In [3]:
train_df = pd.read_csv('Data/TeleoSplit/train_aug_nodupl_300.csv')
val_df = pd.read_csv('Data/TeleoSplit/val_genera.csv')
test_df = pd.read_csv('Data/TeleoSplit/test_genera.csv')

# model


In [13]:
tokeN=KmerTokenizer(3, trust_remote_code=True,add_special_tokens=True)

In [14]:
train_encodings = tokeN(train_df['sequence'].tolist(), truncation=True, max_length=512)
val_encodings = tokeN(val_df['sequence'].tolist(), truncation=True, max_length=512)
test_encodings = tokeN(test_df['sequence'].tolist(), truncation=True, max_length=512)

In [6]:
# Initialize the label encoder
label_encoder = LabelEncoder()

# Fit the label encoder on the combined data
label_encoder.fit(pd.concat([train_df['family'], val_df['family']]))

# Encode the labels
train_labels = label_encoder.transform(train_df['family'])
val_labels = label_encoder.transform(val_df['family'])

In [7]:
train_dataset = Dataset.from_dict({
    'input_ids': train_encodings['input_ids'],
    'attention_mask': train_encodings['attention_mask'],
    'labels': train_labels
})

val_dataset = Dataset.from_dict({
    'input_ids': val_encodings['input_ids'],
    'attention_mask': val_encodings['attention_mask'],
    'labels': val_labels
})

test_dataset = Dataset.from_dict({
    'input_ids': test_encodings['input_ids'],
    'attention_mask': test_encodings['attention_mask'],
    'labels': label_encoder.transform(test_df['family'])
})

In [8]:
# Create dictionnaries for the class labels
id2label = {i: label for i, label in enumerate(label_encoder.classes_)}
label2id = {label: i for i, label in enumerate(label_encoder.classes_)}

In [9]:
data_collator = DataCollatorWithPadding(tokenizer=tokeN)

# Train

In [10]:
config = BertConfig.from_pretrained("zhihan1996/DNABERT-2-117M", 
                                    num_labels=len(id2label), 
                                    max_position_embeddings=514,
                                    id2label=id2label,
                                    label2id=label2id
)

model = AutoModelForSequenceClassification.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True, ignore_mismatched_sizes=True, config=config)

# Verif that the model is correctly configured
model.id2label = id2label
model.label2id = label2id

C:\Users\Auguste Verdier\.cache\huggingface\modules\transformers_modules\zhihan1996\DNABERT-2-117M\d064dece8a8b41d9fb8729fbe3435278786931f1\bert_layers.py:129: UserWarning: Unable to import Triton; defaulting MosaicBERT attention implementation to pytorch (this will reduce throughput when using this model).
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at zhihan1996/DNABERT-2-117M and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
# Update the trainer arguments to use your datasets
training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=15,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",  # Use F1 score to select the best model
    greater_is_better=True,      # Higher F1 score is better
    push_to_hub=False,
)

# Load F1 metric
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred.predictions, eval_pred.label_ids


    if isinstance(predictions, tuple):
        predictions = predictions[0]
    # Ensure predictions are 2D
    if predictions.ndim == 2:
        predictions = np.argmax(predictions, axis=1)
    elif predictions.ndim > 2:
        raise ValueError(f"Unexpected predictions shape: {predictions.shape}")

    return f1_metric.compute(predictions=predictions, references=labels, average='macro')

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokeN,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [13]:
trainer.train()

  0%|          | 0/85335 [00:00<?, ?it/s]

{'loss': 5.6944, 'grad_norm': 2.164126396179199, 'learning_rate': 9.94140739438683e-06, 'epoch': 0.09}
{'loss': 5.5746, 'grad_norm': 2.61074161529541, 'learning_rate': 9.882814788773658e-06, 'epoch': 0.18}
{'loss': 5.4275, 'grad_norm': 2.668440341949463, 'learning_rate': 9.824222183160485e-06, 'epoch': 0.26}
{'loss': 5.2633, 'grad_norm': 3.4311981201171875, 'learning_rate': 9.765629577547314e-06, 'epoch': 0.35}
{'loss': 5.0965, 'grad_norm': 3.194411277770996, 'learning_rate': 9.707036971934143e-06, 'epoch': 0.44}
{'loss': 4.925, 'grad_norm': 3.1747934818267822, 'learning_rate': 9.648444366320971e-06, 'epoch': 0.53}
{'loss': 4.7501, 'grad_norm': 4.537007808685303, 'learning_rate': 9.589851760707798e-06, 'epoch': 0.62}
{'loss': 4.5707, 'grad_norm': 3.4933345317840576, 'learning_rate': 9.531259155094627e-06, 'epoch': 0.7}
{'loss': 4.4166, 'grad_norm': 3.5773255825042725, 'learning_rate': 9.472666549481456e-06, 'epoch': 0.79}
{'loss': 4.2597, 'grad_norm': 6.921662330627441, 'learning_rate'

  0%|          | 0/77 [00:00<?, ?it/s]

Predictions type: <class 'tuple'>
Predictions shape after handling tuple: (1226, 303)
Labels shape: (1226,)
{'eval_loss': 4.163370609283447, 'eval_f1': 0.09266847644652851, 'eval_runtime': 2.6292, 'eval_samples_per_second': 466.295, 'eval_steps_per_second': 29.286, 'epoch': 1.0}
{'loss': 3.9664, 'grad_norm': 4.587470054626465, 'learning_rate': 9.296888732641942e-06, 'epoch': 1.05}
{'loss': 3.846, 'grad_norm': 4.607370376586914, 'learning_rate': 9.23829612702877e-06, 'epoch': 1.14}
{'loss': 3.7246, 'grad_norm': 5.129832744598389, 'learning_rate': 9.179703521415597e-06, 'epoch': 1.23}
{'loss': 3.6164, 'grad_norm': 6.261531829833984, 'learning_rate': 9.121110915802426e-06, 'epoch': 1.32}
{'loss': 3.5032, 'grad_norm': 5.075292110443115, 'learning_rate': 9.062518310189255e-06, 'epoch': 1.41}
{'loss': 3.4063, 'grad_norm': 5.411041259765625, 'learning_rate': 9.003925704576084e-06, 'epoch': 1.49}
{'loss': 3.3114, 'grad_norm': 5.023280143737793, 'learning_rate': 8.945333098962912e-06, 'epoch': 

  0%|          | 0/77 [00:00<?, ?it/s]

...

  0%|          | 0/77 [00:00<?, ?it/s]

Predictions type: <class 'tuple'>
Predictions shape after handling tuple: (1226, 303)
Labels shape: (1226,)
{'eval_loss': 1.640536904335022, 'eval_f1': 0.40964526908049725, 'eval_runtime': 2.203, 'eval_samples_per_second': 556.513, 'eval_steps_per_second': 34.952, 'epoch': 14.0}
{'loss': 0.2588, 'grad_norm': 17.287996292114258, 'learning_rate': 6.251831018925412e-07, 'epoch': 14.06}
{'loss': 0.2546, 'grad_norm': 10.865702629089355, 'learning_rate': 5.665904962793696e-07, 'epoch': 14.15}
{'loss': 0.27, 'grad_norm': 10.387070655822754, 'learning_rate': 5.07997890666198e-07, 'epoch': 14.24}
{'loss': 0.2651, 'grad_norm': 20.386409759521484, 'learning_rate': 4.494052850530264e-07, 'epoch': 14.33}
{'loss': 0.2661, 'grad_norm': 21.443925857543945, 'learning_rate': 3.908126794398547e-07, 'epoch': 14.41}
{'loss': 0.2574, 'grad_norm': 12.499202728271484, 'learning_rate': 3.322200738266831e-07, 'epoch': 14.5}
{'loss': 0.2627, 'grad_norm': 11.487359046936035, 'learning_rate': 2.736274682135115e-07

  0%|          | 0/77 [00:00<?, ?it/s]

Predictions type: <class 'tuple'>
Predictions shape after handling tuple: (1226, 303)
Labels shape: (1226,)
{'eval_loss': 1.6489739418029785, 'eval_f1': 0.4118059703854988, 'eval_runtime': 2.227, 'eval_samples_per_second': 550.516, 'eval_steps_per_second': 34.576, 'epoch': 15.0}
{'train_runtime': 15693.416, 'train_samples_per_second': 86.993, 'train_steps_per_second': 5.438, 'train_loss': 1.273059845909458, 'epoch': 15.0}


TrainOutput(global_step=85335, training_loss=1.273059845909458, metrics={'train_runtime': 15693.416, 'train_samples_per_second': 86.993, 'train_steps_per_second': 5.438, 'total_flos': 6.580211547141181e+16, 'train_loss': 1.273059845909458, 'epoch': 15.0})

# Val

## Load model

In [10]:
model_path_save=r"C:\Users\Auguste Verdier\Desktop\ADNe\BouillaClip\Model\genera_300_medium_3_mer\checkpoint-85335"
config = BertConfig.from_pretrained(os.path.join(model_path_save,"config.json"), 
                                    num_labels=len(id2label), 
                                    max_position_embeddings=510,
                                    id2label=id2label,
                                    label2id=label2id
)

model = AutoModelForSequenceClassification.from_pretrained(model_path_save, trust_remote_code=True, ignore_mismatched_sizes=True, config=config).to(device)


C:\Users\Auguste Verdier\.cache\huggingface\modules\transformers_modules\zhihan1996\DNABERT-2-117M\d064dece8a8b41d9fb8729fbe3435278786931f1\bert_layers.py:126: UserWarning: Unable to import Triton; defaulting MosaicBERT attention implementation to pytorch (this will reduce throughput when using this model).
  warnings.warn(


In [11]:
# Update the trainer arguments to use your datasets
training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=15,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",  # Use F1 score to select the best model
    greater_is_better=True,      # Higher F1 score is better
    push_to_hub=False,
)

# Load F1 metric
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred.predictions, eval_pred.label_ids


    if isinstance(predictions, tuple):
        predictions = predictions[0]
    # Ensure predictions are 2D
    if predictions.ndim == 2:
        predictions = np.argmax(predictions, axis=1)
    elif predictions.ndim > 2:
        raise ValueError(f"Unexpected predictions shape: {predictions.shape}")

    return f1_metric.compute(predictions=predictions, references=labels, average='macro')

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokeN,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [12]:
trainer.predict(test_dataset)

  0%|          | 0/154 [00:00<?, ?it/s]

PredictionOutput(predictions=(array([[-2.7792404 ,  1.0934638 , -4.942048  , ...,  0.66068083,
         7.4447284 , -1.5372963 ],
       [-3.296713  , -0.5567811 , -1.4588093 , ..., -2.433885  ,
         1.7861667 , -0.20132664],
       [ 1.4642676 , -1.8723526 ,  1.354167  , ...,  1.9308411 ,
        -1.8681207 , -0.7271986 ],
       ...,
       [ 1.0890701 ,  3.3553426 ,  0.10366318, ...,  3.478224  ,
        -5.1953177 , -1.7568536 ],
       [ 1.7054284 ,  0.6383738 ,  3.1199536 , ..., -1.1823521 ,
        -3.8631444 , -1.5513173 ],
       [ 1.4084353 , -1.34002   , -2.723738  , ..., -2.0261855 ,
         0.36796463,  4.4732676 ]], dtype=float32), array([[[ 3.51176798e-01,  6.74668074e-01, -4.27358449e-01, ...,
          4.79375094e-01,  1.31157160e-01, -1.76985841e-02],
        [ 6.49972618e-01,  6.87060833e-01, -2.50774324e-01, ...,
          9.11989212e-01, -1.39739126e-01, -9.09518301e-02],
        [ 5.99686861e-01,  6.87872887e-01, -2.56467890e-02, ...,
          1.01779830e+00